# Import Widget Libraries

In [ ]:
%load_ext autoreload   
%autoreload 2
import kyurem
from kyurem.core.idom_loader import reload_bundle
from kyurem.core.widget import WidgetModel

# Import Graph Querying Library

In [ ]:
from kyurem import Service
service = Service(kh = 'indeed.v0.0.1:612e3403')

In [ ]:
schema = service.get_kh_edge_list()

# Create an Overview/Exploration Widget

In [ ]:
from kyurem import Explorer


def init(state): 
    # Fetch initial view data
    return {
        "subgraph": None,
        "children": service.get_children_node_distributions(),
        "relations": service.get_relation_distribution(),
    }

def focus(state, node, panel):
    # node can be in one of two formats:
    # node : { "node_label": str, "node_property": str, "node_property_value": str }
    # node : { "node_label": str, "title": str, "uuid": str }
    
    # Since the service functions use the first format, node must first
    # be converted to the proper format
    if "title" in node:
        node = {
            "node_label": node["node_label"],
            "node_property": "title",
            "node_property_value": node["title"]
        }
        
    # TODO: Return nodes in a consistent format from the service
    #       functions to streamline the above
    
    
    # Fetch the neighborhood around the input node 
    result = service.get_node_neighborhood(node)
    
    data = { "subgraph": None }
    if panel != "schema":
        data["schema"] = result["schema"]
    if panel != "children":
        # TODO: Accept a consistent input format (e.g., same as get_node_neighborhood)
        data["children"] = service.get_children_node_distributions(
            node
        )

    # Use a list comprehension to convert the return format of the relation_dist 
    # to a bar-chart format
    data["relations"] = [
        {"x": relation["label"], "y": relation["count"], "type": type}
        for type, relations in result["relation_dist"].items()
        for relation in relations
    ]
    # TODO: Move conversion code into service
    
    return data
  
# Create widget 
reload_bundle()
explorer = Explorer({
        "init": init,
        "focus": focus
    }, schema)

explorer.show()

In [ ]:
from kyurem.utils.mdump import mdump
print(mdump(explorer.history, 3))

In [ ]:
# def filter_by_relation(state, type, direction):
#     if state.nodelabel and state.nodetitle:
#         node = {
#             "node_label": state.nodelabel,
#             "node_property": "title",
#             "node_property_value": state.nodetitle,
#         }
#     else:
#         node = None

#     result = service.get_relation_neighborhood(node, {"type": type, "direction": direction})
#     data = {}
#     data['schema'] = result["schema"]
#     if not node:
#         data['children'] = [
#             {"x": x, "y": y} for x, y in result["node_dist"].items()
#         ]
#     return data

# View 0: Send nodes rather than labels in schema

In [ ]:
schema

# View 0: Click a node on minimap

In [ ]:
node ={'node_label': 'occupation',
       'node_property': 'title', 
       'node_property_value': 'occupation'}

service.get_children_node_distributions('occupation', 'title', 'Healthcare Occupations')

# View 0: Click a bar on node_dist

In [ ]:
node ={'node_label': 'occupation',
       'node_property': 'title', 
       'node_property_value': 'Allied Health Occupations'}

In [ ]:
nn = service.get_node_neighborhood(node)
nn

# View 0: Click a bar on relation dist

In [ ]:
node ={'node_label': 'occupation',
       'node_property': 'title', 
       'node_property_value': 'Fire Alarm Technicians'}

relation = {}
relation['type'] = 'has_core_skill'
relation['direction'] = 'out'

In [ ]:
service.get_relation_neighborhood(node, relation)

In [ ]:
service.get_relation_neighborhood(None, relation)

# View 1: Click a node on zoomed in graph

# View 1: Click a node on node_dist

# View 1: Click a node on relation dist